# Initialization, Making the Preprocessor, and Making the Processed Training File

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import re
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')

In [ ]:
# Save and read data files from your GoogleDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load training .csv file
old_training_doc_df = pd.read_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/topic_model_train_new.csv')

In [ ]:
spacy_lemma = spacy.load("en_core_web_sm")
def my_preprocessor(text):
  """
  Parameters:
    text: (str)

  Changes:
    Converts text to lowercase
    Removed Markdown code
    Removed numbers
    Removed stop words (english and spanish)
    Removed parentheses (and everything inside them)
    Removed punctuation
    Lemmatizes

  """

  # Makes text lowercase
  text_lower = text.lower()

  # Remove parentheses and anything inside them
  text_paren = re.sub(r'\(.*?\)', '', text_lower)

  # Remove numbers
  text_num = re.sub(r'\d+', '', text_paren)

  # Remove markdown / LaTeX code (starts with '\' or '$' or contain underscores)
  text_clean = re.sub(r'[\$\\]\S+|\b\w*_\w*\b', '', text_num)

  # Split text into words (also gets rid of punctuation)
  tokens = RegexpTokenizer(r'\w+').tokenize(text_clean)

  # Define stop words
  stop_words = set(stopwords.words('english')) | set(stopwords.words('spanish'))

  # Removes stopwords
  stopunct_tokens = []
  for token in tokens:
    if token not in stop_words:
      stopunct_tokens.append(token)

  # Combines text for and processes with SpaCy
  text_processed_0 = ' '.join(stopunct_tokens)
  text_spacy = spacy_lemma(text_processed_0)

  # Lemmatization
  pos_tags = {'NOUN', 'ADJ', 'VERB', 'ADV'}
  lemma_tokens = []
  for token in text_spacy:
    if token.pos_ in pos_tags:
      lemma_tokens.append(token.lemma_)

  common_words = [
      'approach',
      'consider',
      'define',
      'different',
      'feature',
      'first',
      'general',
      'however',
      'known',
      'method',
      'network',
      'number',
      'obtain',
      'present',
      'problem',
      'propose',
      'provide',
      'result'
  ]

  # Keep words > 4 letters and rid of common words with little meaning
  final_tokens = []
  for token in lemma_tokens:
    if len(token) > 5 and token not in common_words:
      final_tokens.append(token)

  # Make one string again
  text_processed = ' '.join(final_tokens)

  return text_processed

In [ ]:
# Make and save the processed training file (if doesn't exist yet)
file_path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/training_processed.csv'
if os.path.exists(file_path):
  train_df = pd.read_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/training_processed.csv')
  print('File Already Exists. Continue.')
else:
  train_df = old_training_doc_df.copy()
  train_df['ABSTRACT'] = train_df['ABSTRACT'].apply(my_preprocessor)
  path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/'
  train_df.to_csv(f'{path}training_processed.csv', index=False)

In [ ]:
# Show a histogram of the word count per abstract
train_df['ABSTRACT_word_count'] = train_df['ABSTRACT'].apply(lambda x: len(str(x).split(" ")))
plt.hist(train_df['ABSTRACT_word_count'])
plt.xlabel("Words per Abstract")
plt.ylabel("Abstracts")
plt.show()

In [ ]:
# Look at two examples of processed vs. not processed
for i in range(2): # Use to see first Abstract
    abstract = old_training_doc_df['ABSTRACT'].iloc[i]
    print(f'Abstract {i+1} (Not Processed):', abstract)
    abstract = train_df['ABSTRACT'].iloc[i]
    print(f'Abstract {i+1} (Processed):', abstract, '\n')

# Sentence Embedding (SBERT - "allenai-specter") on the Training Set

### Initialization

In [ ]:
!pip install -U sentence-transformers
import scipy.cluster.hierarchy as sch
from nltk.tokenize import word_tokenize
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

### Making the SBERT Model

In [ ]:
specter_sbert_model = SentenceTransformer("allenai-specter")

In [ ]:
# Make and save the SBERT embeddings (if doesn't exist yet)
embeddings_path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/train_sbert_embeddings.npy'
if os.path.exists(embeddings_path):
  print('(Training) SBERT Embeddings already exist. Continue.')
  train_sbert_embeddings = np.load(embeddings_path)
else:
  train_sbert_embeddings = specter_sbert_model.encode(train_df['ABSTRACT'], batch_size = 32, show_progress_bar=True)
  np.save(embeddings_path, train_sbert_embeddings)

### Testing Query

In [ ]:
# Testing toy search engine with example query
query = 'Artifical Neural Networks Programming Code'
query_embedding = specter_sbert_model.encode(query)

In [ ]:
# Cosine similarity
cosine_similarities = cosine_similarity(query_embedding.reshape(1, -1), train_sbert_embeddings)
cosine_similarities = cosine_similarities[0]

# Dot product
dot_products = np.dot(train_sbert_embeddings, query_embedding)

# Euclidean distance
euclidean_distances = np.array([distance.euclidean(embedding, query_embedding) for embedding in train_sbert_embeddings])

# Normalize all scores to get an average
normalized_cosine = (cosine_similarities - np.min(cosine_similarities)) / (np.max(cosine_similarities) - np.min(cosine_similarities))
normalized_dot = (dot_products - np.min(dot_products)) / (np.max(dot_products) - np.min(dot_products))
normalized_euclidean = 1 - (euclidean_distances - np.min(euclidean_distances)) / (np.max(euclidean_distances) - np.min(euclidean_distances))

# Add scores to DataFrame
old_training_doc_df['Cosine Similarity'] = cosine_similarities
old_training_doc_df['Dot Product'] = dot_products
old_training_doc_df['Euclidean Distance'] = euclidean_distances
old_training_doc_df['Geometric Mean'] = (normalized_cosine * normalized_dot * normalized_euclidean)**(1/3)

# Sort and display top results by geometric mean
top_10_results = old_training_doc_df.sort_values(by='Geometric Mean', ascending=False).head(10)
top_10_results

# Preprocessing the Test Data Set

In [ ]:
# Make and save the processed training file (if doesn't exist yet)
file_path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv'
if os.path.exists(file_path):
  test_df = pd.read_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv')
  print('File Already Exists. Continue.')
  old_test_df = pd.read_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/topic_model_test_new.csv').copy()
else:
  old_test_df = pd.read_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/topic_model_test_new.csv').copy()
  test_df['ABSTRACT'] = old_test_df['ABSTRACT'].apply(my_preprocessor)
  path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/'
  test_df.to_csv(f'{path}test_processed.csv', index=False)

In [ ]:
# Show a Histogram of the word count per abstract
test_df['ABSTRACT_word_count'] = test_df['ABSTRACT'].apply(lambda x: len(str(x).split(" ")))
plt.hist(test_df['ABSTRACT_word_count'])
plt.xlabel("Words per Abstract")
plt.ylabel("Abstracts")
plt.show()

In [ ]:
# Look at two examples of processed vs. not processed
for i in range(2): # Use to see first Abstract
    abstract = old_test_df['ABSTRACT'].iloc[i]
    print(f'Abstract {i+1} (Not Processed):', abstract)
    abstract = test_df['ABSTRACT'].iloc[i]
    print(f'Abstract {i+1} (Processed):', abstract, '\n')

# Sentence Embedding (SBERT - "allenai-specter" and SBERT - "all-MiniLM-L6-v2" ) on Testing Sets

### Function to Find Top Document and Rank

In [ ]:
def find_top_doc(query, model, embeddings, doc_df, true_id):

  # Decide on model
  if model == 'specter':
    query_embedding = specter_sbert_model.encode(query)
  elif model == 'minilm':
    query_embedding = minilm_sbert_model.encode(query)
  else:
    print('Enter "specter" for allenai-specter and "minilm" for all-MiniLM-L6-v2')

  # Cosine similarity
  cosine_similarities = cosine_similarity(query_embedding.reshape(1, -1), embeddings)
  cosine_similarities = cosine_similarities[0]

  # Dot product
  dot_products = np.dot(embeddings, query_embedding)

  # Euclidean distance
  euclidean_distances = np.array([distance.euclidean(embedding, query_embedding) for embedding in embeddings])

  # Normalize all scores to get an average
  normalized_cosine = (cosine_similarities - np.min(cosine_similarities)) / (np.max(cosine_similarities) - np.min(cosine_similarities))
  normalized_dot = (dot_products - np.min(dot_products)) / (np.max(dot_products) - np.min(dot_products))
  normalized_euclidean = 1 - (euclidean_distances - np.min(euclidean_distances)) / (np.max(euclidean_distances) - np.min(euclidean_distances))
  doc_df['Geometric Mean'] = (normalized_cosine * normalized_dot * normalized_euclidean)**(1/3)

  # Sort by geometric mean
  sorted_df = doc_df.sort_values(by='Geometric Mean', ascending=False)

  # Get top doc ID
  top_doc_id = sorted_df.iloc[0]['ID']

  # Determine rank of original topic
  rank_doc_id = np.where(sorted_df['ID'] == true_id)[0][0] + 1

  return top_doc_id, rank_doc_id

### "all-MiniLM-L6-v2" Model

In [ ]:
# Make the model
minilm_sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Make and save the SBERT embeddings (if doesn't exist yet)
embeddings_path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/minilm_test_sbert_embeddings.npy'
if os.path.exists(embeddings_path):
  print('MiniLM (test) SBERT Embeddings already exist. Continue.')
  minilm_sbert_embeddings = np.load(embeddings_path)
else:
  minilm_sbert_embeddings = minilm_sbert_model.encode(test_df['ABSTRACT'], batch_size = 32, show_progress_bar=True)
  np.save(embeddings_path, minilm_sbert_embeddings)

In [ ]:
# Save ID's so it doesn't have to be run again
if 'MiniLM Predicted ID' not in test_df.columns or 'MiniLM Rank' not in test_df.columns:
  minilm_ids = []
  minilm_ranks = []

  # Place expected ID and true rank in column
  for title in range(test_df.shape[0]):
    test_id = test_df['ID'].iloc[title]
    query = test_df['TITLE'].iloc[title]
    query_embedding = minilm_sbert_model.encode(query)
    minilm_id_placeholder, minilm_rank_placeholder = find_top_doc(query, 'minilm', minilm_sbert_embeddings, test_df, test_id)
    minilm_ids.append(minilm_id_placeholder)
    minilm_ranks.append(minilm_rank_placeholder)


  test_df['MiniLM Predicted ID'] = minilm_ids
  test_df['MiniLM Rank'] = minilm_ranks
  test_df.to_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv', index=False)
else:
  print('Predicted MiniLM ID and rank columns exist.')

### "allenai-specter" Model

In [ ]:
# Make the model
specter_sbert_model = SentenceTransformer("allenai-specter")

In [ ]:
# Make and save the SBERT embeddings (if doesn't exist yet)
embeddings_path = '/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/specter_test_sbert_embeddings.npy'
if os.path.exists(embeddings_path):
  print('Specter (test) SBERT Embeddings already exist. Continue.')
  specter_sbert_embeddings = np.load(embeddings_path)
else:
  specter_sbert_embeddings = specter_sbert_model.encode(test_df['ABSTRACT'], batch_size = 32, show_progress_bar=True)
  np.save(embeddings_path, specter_sbert_embeddings)

In [ ]:
# Save ID's so it doesn't have to be run again
if 'Specter Predicted ID' not in test_df.columns or 'Specter Rank' not in test_df.columns:
  specter_ids = []
  specter_ranks = []

  # Place expected ID in column
  for title in range(test_df.shape[0]):
    test_id = test_df['ID'].iloc[title]
    query = test_df['TITLE'].iloc[title]
    query_embedding = specter_sbert_model.encode(query)
    specter_id_placeholder, specter_rank_placeholder = find_top_doc(query, 'specter', specter_sbert_embeddings, test_df, test_id)
    specter_ids.append(specter_id_placeholder)
    specter_ranks.append(specter_rank_placeholder)

  test_df['Specter Predicted ID'] = specter_ids
  test_df['Specter Rank'] = specter_ranks
  test_df.to_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv', index=False)
else:
  print('Predicted Specter ID and rank columns exist.')

### Determine Average Recall@1 and MRR From Saved Data, Delete Geometric Mean Column and Show Full Table

In [ ]:
# Determine Average Recall@1 and MRR
specter_recall_count = 0
specter_mrr_sum = 0
minilm_recall_count = 0
minilm_mrr_sum = 0
for row in range(len(test_df)):
  if test_df['ID'].iloc[row] == test_df['Specter Predicted ID'].iloc[row]:
    specter_recall_count += 1
  specter_mrr_sum += 1/(test_df['Specter Rank'].iloc[row])
  if test_df['ID'].iloc[row] == test_df['MiniLM Predicted ID'].iloc[row]:
    minilm_recall_count += 1
  minilm_mrr_sum += 1/(test_df['MiniLM Rank'].iloc[row])

specter_ave_recall = specter_recall_count / test_df.shape[0]
specter_mrr = specter_mrr_sum / test_df.shape[0]
minilm_ave_recall = minilm_recall_count / test_df.shape[0]
minilm_mrr = minilm_mrr_sum / test_df.shape[0]

print(f'Specter Average Recall@1: {specter_ave_recall:.4f}')
print(f'Specter MRR: {specter_mrr:.4f}')
print(f'Minilm Average Recall@1: {minilm_ave_recall:.4f}')
print(f'Minilm MRR: {minilm_mrr:.4f}')

In [ ]:
# Delete Geometric Mean Column and save file
if 'Geometric Mean' in test_df.columns:
  test_df = test_df.drop(['Geometric Mean'], axis=1)
test_df.to_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv', index=False)

# TF-IDF and TF on Testing Sets

### Initialization

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

### TF-IDF

In [ ]:
# Fill NaN values with an empty string
train_df['ABSTRACT'] = train_df['ABSTRACT'].fillna('')
test_df['ABSTRACT'] = test_df['ABSTRACT'].fillna('')

# Make vectorizer
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    sublinear_tf=True,
    use_idf=True,
    smooth_idf=True,
)

# Fit to training data
tfidf_vectorizer.fit(train_df['ABSTRACT'])

# Transform on test data
tfidf_test = tfidf_vectorizer.transform(test_df['ABSTRACT'])

# Transform test titles (queries)
tfidf_queries = tfidf_vectorizer.transform(test_df['TITLE'])

In [ ]:
# Fill NaN values with# Save ID's so it doesn't have to be run again
if 'TFIDF Predicted ID' not in test_df.columns or 'TFIDF Rank' not in test_df.columns:
  tfidf_ids = []
  tfidf_rank = []
  # Place expected ID and rank of actual ID in column
  for title in range(test_df.shape[0]):
  # for title in range(5):
    test_id = test_df['ID'].iloc[title]
    query = tfidf_queries[title]
    sim_scores = cosine_similarity(query, tfidf_test).flatten()
    best_index = sim_scores.argmax()
    predicted_id = test_df['ID'].iloc[best_index]

    sorted_indices = sim_scores.argsort()[::-1]
    sorted_ids = test_df['ID'].iloc[sorted_indices].tolist()
    rank = sorted_ids.index(test_id) + 1  # 1-based rank

    tfidf_ids.append(predicted_id)
    tfidf_rank.append(rank)

  test_df['TFIDF Predicted ID'] = tfidf_ids
  test_df['TFIDF Rank'] = tfidf_rank
  test_df.to_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv', index=False)
else:
  print('Predicted TFIDF ID and Rank Columns exist.')

### TF

In [ ]:
# Fill NaN values with an empty string
train_df['ABSTRACT'] = train_df['ABSTRACT'].fillna('')
test_df['ABSTRACT'] = test_df['ABSTRACT'].fillna('')

# Make vectorizer
tf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    sublinear_tf=True,
    use_idf=False,
    norm=None,
)

# Fit to training data
tf_vectorizer.fit(train_df['ABSTRACT'])

# Transform on test data
tf_test = tf_vectorizer.transform(test_df['ABSTRACT'])

# Transform test titles (queries)
tf_queries = tf_vectorizer.transform(test_df['TITLE'])

In [ ]:
# Save ID's so it doesn't have to be run again
if 'TF Predicted ID' not in test_df.columns or 'TF Rank' not in test_df.columns:
  tf_ids = []
  tf_rank = []
  # Place expected ID and rank of actual ID in column
  for title in range(test_df.shape[0]):
  # for title in range(5):
    test_id = test_df['ID'].iloc[title]
    query = tf_queries[title]
    sim_scores = cosine_similarity(query, tf_test).flatten()
    best_index = sim_scores.argmax()
    predicted_id = test_df['ID'].iloc[best_index]

    sorted_indices = sim_scores.argsort()[::-1]
    sorted_ids = test_df['ID'].iloc[sorted_indices].tolist()
    rank = sorted_ids.index(test_id) + 1  # 1-based rank

    tf_ids.append(predicted_id)
    tf_rank.append(rank)

  test_df['TF Predicted ID'] = tf_ids
  test_df['TF Rank'] = tf_rank
  test_df.to_csv('/content/drive/MyDrive/2024 Spring/Text Mining/Projects/Project3/test_processed.csv', index=False)
else:
  print('Predicted TF ID and Rank Columns exist.')

### Determine Average Recall@1 and MRR From Saved Data

In [ ]:
# Determine Average Recall@1 and MRR
tf_recall_count = 0
tf_mrr_sum = 0
tfidf_recall_count = 0
tfidf_mrr_sum = 0
for row in range(len(test_df)):
  if test_df['ID'].iloc[row] == test_df['TF Predicted ID'].iloc[row]:
    tf_recall_count += 1
  tf_mrr_sum += 1/(test_df['TF Rank'].iloc[row])
  if test_df['ID'].iloc[row] == test_df['TF Predicted ID'].iloc[row]:
    tfidf_recall_count += 1
  tfidf_mrr_sum += 1/(test_df['TFIDF Rank'].iloc[row])

tf_ave_recall = tf_recall_count / test_df.shape[0]
tf_mrr = tf_mrr_sum / test_df.shape[0]
tfidf_ave_recall = tfidf_recall_count / test_df.shape[0]
tfidf_mrr = tfidf_mrr_sum / test_df.shape[0]

print(f'TF Average Recall@1: {tf_ave_recall:.4f}')
print(f'TF MRR: {tf_mrr:.4f}')
print(f'TFIDF Average Recall@1: {tfidf_ave_recall:.4f}')
print(f'TFIDF MRR: {tfidf_mrr:.4f}')

# Final Table and Results

In [ ]:
print(f'TF Average Recall@1: {tf_ave_recall:.4f}')
print(f'TF MRR: {tf_mrr:.4f}')
print(f'TFIDF Average Recall@1: {tfidf_ave_recall:.4f}')
print(f'TFIDF MRR: {tfidf_mrr:.4f}')
print(f'Specter Average Recall@1: {specter_ave_recall:.4f}')
print(f'Specter MRR: {specter_mrr:.4f}')
print(f'MiniLM Average Recall@1: {minilm_ave_recall:.4f}')
print(f'MiniLM MRR: {minilm_mrr:.4f}')

In [ ]:
test_df